In [1]:
from genetic_algorithm import *
import re
import json
import numpy as np
from collections import Counter
import copy

## hexadecimal generator

In [2]:
nr_of_input = 3
nr_of_actions = 3
nr_of_inner = 2
nr_of_genes = 16
nr_individuals = 20

individuals = generate_initial_genomes_for_population(nr_individuals, nr_of_genes, nr_of_input, nr_of_actions, nr_of_inner)

In [12]:
## sum duplicates
individuals_sum_dup = sum_duplicated_neurons(individuals)

## remove self loops
individuals_sum_dup_no_self_loop = remove_self_loop(individuals_sum_dup)

## brain generator
for individual in individuals_sum_dup_no_self_loop:
#     try:
    edges = individuals_sum_dup_no_self_loop[individual]
    edges = [tuple(edges[i]) for i in edges]

    ## preprocessing
    remove_mid_with_no_predecessor(edges) 
    dic = weight_sum_preprocessing(edges)

    ## calculate weight sum
    mid_to_weight(dic)
    out_to_weight(dic)
       
    dic = {k:v for k, v in dic.items() if 'out' in k}    
    print(json.dumps(dic, indent=4, sort_keys=True))
    print()

{
    "out0": -0.9884078125303537,
    "out1": -0.024533525331122213,
    "out2": 0.9999958776168677
}

{
    "out0": 0.990212978082521,
    "out1": 0.18688186258306475,
    "out2": -0.7154683334692711
}

{
    "out0": 0.9707445912193801,
    "out1": -0.9062512248140372,
    "out2": 0.9999465053683078
}

{
    "out0": -0.9995595383453032,
    "out1": -0.7256442035892058,
    "out2": -0.9991728549445408
}

{
    "out0": -0.7166295074933271,
    "out1": 0.9922863514483884,
    "out2": 0.9911279204224654
}

{
    "out1": -0.7060798213246395,
    "out2": 0.9976642696956707
}

{
    "out0": -0.9985415944465937,
    "out1": 0.9974827632002929,
    "out2": -0.37388850638715293
}

{
    "out0": -0.9732186436726274,
    "out1": 0.99998466736118,
    "out2": -0.9997679230866032
}

{
    "out0": -0.994024807806663,
    "out1": 0.9793372597238998,
    "out2": -0.9999839494749168
}

{
    "out0": -0.7814352787011383,
    "out2": -0.9999480668342098
}

{
    "out0": 0.9954545371171831,
    "out1": -